In [50]:
import pandas as pd
import numpy as np

import time
import pickle
from pathlib import Path

In [51]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [52]:
# FIX VSCODE PATH PROBLEM -- DELETE AFTER
import sys

# add path
sys.path.append(
    "/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/"
)
print(sys.path)

['/gpfs01/berens/user/rgonzalesmarquez', '/.pyenv/versions/miniconda3-latest/lib/python311.zip', '/.pyenv/versions/miniconda3-latest/lib/python3.11', '/.pyenv/versions/miniconda3-latest/lib/python3.11/lib-dynload', '', '/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages', '/gpfs01/berens/user/rgonzalesmarquez/phd/text-embeddings', '/gpfs01/berens/user/rgonzalesmarquez/phd/pubmed-landscape', '/.pyenv/versions/miniconda3-latest/lib/python3.11/site-packages', '/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/', '/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/']


In [54]:
%load_ext autoreload
%autoreload 2

from daily_updates_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
variables_path = Path("../results/variables")
figures_path = Path("../figures")
data_path = Path("../data")
berenslab_data_path = Path("/gpfs01/berens/data/data/pubmed_processed")

In [6]:
# MANUAL FIX TO PATH ISSUE FROM VSCODE

nb_path = Path(
    "/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts"
)
assert nb_path.exists(), "The path does not exist"

variables_path = (nb_path / variables_path).resolve(strict=True)
figures_path = (nb_path / figures_path).resolve(strict=True)
data_path = (nb_path / data_path).resolve(strict=True)

We extract from PubMed's metadata `.xml` files the following:
- PubMed ID
- title
- abstract
- language
- journal title
- ISSN
- publication date
- (first and last) author first names
- (first and last) author Affiliations


# Parse data

## 2024 daily updates

In [6]:
%%time

path = "/gpfs01/berens/data/data/pubmed/2024_daily_updates/"

files_2024_df_daily_updates = import_all_files(path, order_files=True)

pubmed24n1220.xml
pubmed24n1221.xml
pubmed24n1222.xml
pubmed24n1223.xml
pubmed24n1224.xml
pubmed24n1225.xml
pubmed24n1226.xml
pubmed24n1227.xml
pubmed24n1228.xml
pubmed24n1229.xml
pubmed24n1230.xml
pubmed24n1231.xml
pubmed24n1232.xml
pubmed24n1233.xml
pubmed24n1234.xml
pubmed24n1235.xml
pubmed24n1236.xml
pubmed24n1237.xml
pubmed24n1238.xml
pubmed24n1239.xml
pubmed24n1240.xml
pubmed24n1241.xml
pubmed24n1242.xml
pubmed24n1243.xml
pubmed24n1244.xml
pubmed24n1245.xml
pubmed24n1246.xml
pubmed24n1247.xml
pubmed24n1248.xml
pubmed24n1249.xml
pubmed24n1250.xml
pubmed24n1251.xml
pubmed24n1252.xml
pubmed24n1253.xml
pubmed24n1254.xml
pubmed24n1255.xml
pubmed24n1256.xml
pubmed24n1257.xml
pubmed24n1258.xml
pubmed24n1259.xml
pubmed24n1260.xml
pubmed24n1261.xml
pubmed24n1262.xml
pubmed24n1263.xml
pubmed24n1264.xml
pubmed24n1265.xml
pubmed24n1266.xml
pubmed24n1267.xml
pubmed24n1268.xml
pubmed24n1269.xml
pubmed24n1270.xml
pubmed24n1271.xml
pubmed24n1272.xml
pubmed24n1273.xml
pubmed24n1274.xml
pubmed24n1

In [7]:
print("There are {} papers".format(files_2024_df_daily_updates.shape[0]))

There are 4129322 papers


In [8]:
files_2024_df_daily_updates.head()

,PMID,Title,AbstractText,Language,Journal,Date,NameFirstAuthor,NameLastAuthor,ISSN,AffiliationFirstAuthor,AffiliationLastAuthor,filename
0,1025,Isolation and purification of cytokinin bindin...,,eng,Biochemical and biophysical research communica...,1975 Nov 17,T,K,0006-291X,,,pubmed24n1220.xml
1,1250,Purification and properties of isoenzymes of c...,Two isoenzymes of an NADP+ -dependent cinnamyl...,eng,European journal of biochemistry,1975 Nov 01,D,H,0014-2956,,,pubmed24n1220.xml
2,433,"Uptake, translocation, and metabolism of tirpa...",,eng,Journal of agricultural and food chemistry,1975 Nov-Dec,J E,R I,0021-8561,,,pubmed24n1220.xml
3,1314,The soybean answer to food costs.,,eng,Dimensions in health service,1976 Jan,T C,T C,0317-7645,,,pubmed24n1220.xml
4,3,Metal substitutions incarbonic anhydrase: a ha...,,eng,Biochemical and biophysical research communica...,1975 Oct 27,R J,R G,0006-291X,,,pubmed24n1220.xml


# Filter

Filter out:
- non-English papers
- papers with empty abstracts
- papers with abstracts shorter than 250 or longer than 4000 symbols
- papers with unfinished abstracts

## Empty abstracts and non-english papers

In [9]:
%%time

print(
    "Before, there were {} papers".format(
        files_2024_df_daily_updates.shape[0]
    )
)

# Eliminate empty abstracts
clean_2024_df_daily_updates = files_2024_df_daily_updates[
    files_2024_df_daily_updates.AbstractText != ""
]

print(
    "After eliminating empty abstracts, there are {} papers".format(
        clean_2024_df_daily_updates.shape[0]
    )
)

# Eliminate non-english papers
clean_2024_df_daily_updates = clean_2024_df_daily_updates[
    clean_2024_df_daily_updates.Language == "eng"
]

# print size
print(
    "After first cleaning, there are {} papers".format(
        clean_2024_df_daily_updates.shape[0]
    )
)

Before, there were 4129322 papers
After eliminating empty abstracts, there are 3712633 papers
After first cleaning, there are 3672467 papers
CPU times: user 4.77 s, sys: 284 ms, total: 5.06 s
Wall time: 5.02 s


## Threshold and cut off

### Cut off = 4000

In [10]:
print(
    "Before cut off, there are {} papers".format(
        clean_2024_df_daily_updates.shape[0]
    )
)
abstracts = clean_2024_df_daily_updates["AbstractText"].tolist()
len_strings = map(len, abstracts)
len_abstracts = np.fromiter(len_strings, dtype=np.int64, count=len(abstracts))


cut_off = 4000
clean_2024_df_daily_updates = clean_2024_df_daily_updates[
    len_abstracts < cut_off
]
print(
    "After cut off, there are {} papers".format(
        clean_2024_df_daily_updates.shape[0]
    )
)

Before cut off, there are 3672467 papers
After cut off, there are 3665725 papers


### Threshold = 250

In [11]:
threshold = 250
len_short_abstracts = len_abstracts[len_abstracts < cut_off]
clean_2024_df_daily_updates = clean_2024_df_daily_updates[
    len_short_abstracts > threshold
]
print(
    "After threshold, there are {} papers".format(
        clean_2024_df_daily_updates.shape[0]
    )
)

After threshold, there are 3640234 papers


## Remove the truncated sentence from abstracts


In [12]:
abstracts = clean_2024_df_daily_updates["AbstractText"]
abstracts_list = clean_2024_df_daily_updates["AbstractText"].tolist()

In [13]:
%%time
# BUENO
clean_2024_df_daily_updates.AbstractText = list(
    map(
        lambda x, y: x[: y - 1] if y != -1 else x,
        clean_2024_df_daily_updates.AbstractText,
        clean_2024_df_daily_updates.AbstractText.str.find(
            "ABSTRACT TRUNCATED AT"
        ),
    )
)

CPU times: user 4.77 s, sys: 122 ms, total: 4.89 s
Wall time: 4.9 s


## Remove unfinished abstracts

In [14]:
abstracts = clean_2024_df_daily_updates["AbstractText"]
abstracts_list = clean_2024_df_daily_updates["AbstractText"].tolist()
print(len(abstracts_list))

3640234


In [15]:
%%time
end_abstracts = [x[-2:] for x in abstracts_list]

CPU times: user 1.15 s, sys: 26.9 ms, total: 1.17 s
Wall time: 1.17 s


In [16]:
%%time
point_index = np.array([x.find(".") for x in end_abstracts])
question_index = np.array([x.find("?") for x in end_abstracts])
exclamation_index = np.array([x.find("!") for x in end_abstracts])

CPU times: user 1.82 s, sys: 2.95 ms, total: 1.82 s
Wall time: 1.83 s


In [17]:
%%time

print(
    "Before cleaning, there are {} papers".format(
        clean_2024_df_daily_updates.shape[0]
    )
)

# Eliminate unfinished abstracts
clean_2024_df_daily_updates = clean_2024_df_daily_updates[
    (point_index != -1) | (question_index != -1) | (exclamation_index != -1)
]

# print size
print(
    "After cleaning, there are {} papers".format(
        clean_2024_df_daily_updates.shape[0]
    )
)

Before cleaning, there are 3640234 papers
After cleaning, there are 3638921 papers
CPU times: user 1.69 s, sys: 41.9 ms, total: 1.73 s
Wall time: 1.73 s


In [110]:
# save intermediate dataframe

# clean_2024_df_daily_updates.to_pickle(
#     data_path / "clean_2024_df_daily_updates_v2"
# )

# Label
Generate labels based on the journal title

In [18]:
label_color_legend = {
    "cancer": "black",
    "neuroscience": "#aeaa00",
    "cardiology": "#1CE6FF",
    "ecology": "#FF34FF",
    "bioinformatics": "#FF4A46",
    "chemistry": "#008941",
    "surgery": "#006FA6",
    "environment": "#0089A3",
    "material": "#0000A6",
    "microbiology": "#B79762",
    "pediatric": "#004D43",
    "immunology": "#8FB0FF",
    "psychology": "#5A0007",
    "psychiatry": "#BA0900",
    "genetics": "#1B4400",
    "nutrition": "#4FC601",
    "veterinary": "#3B5DFF",
    "engineering": "#00C2A0",
    "education": "#549E79",
    "physics": "#BC23FF",
    "optics": "#C895C5",
    "nursing": "#FF2F80",
    "neurology": "#009271",
    "radiology": "#00FECF",
    "ophthalmology": "#A4E804",
    "gynecology": "#FFB500",
    "rehabilitation": "#6B002C",
    "pathology": "#FF9408",
    "anesthesiology": "#CC0744",
    "dermatology": "#D790FF",
    "pharmacology": "#5B4534",
    "physiology": "#E83000",
    "virology": "#6F0062",
    "biochemistry": "#b65141",
    "computation": "#C20078",
    "infectious": "#7A4900",
    "healthcare": "#FF90C9",
    "ethics": "#6508ba",
    "dentistry": "#8e4d8a",
}

In [19]:
%%time
labels_2024_du, _ = improved_coloring(
    clean_2024_df_daily_updates.Journal, label_color_legend
)

CPU times: user 3min 32s, sys: 3.16 s, total: 3min 36s
Wall time: 3min 36s


# Country
Extract first author's affiliation country

In [20]:
all_countries = [
    "Afghanistan",
    "Albania",
    "Algeria",
    "Andorra",
    "Angola",
    "Antigua and Barbuda",
    "Argentina",
    "Armenia",
    "Australia",
    "Austria",
    "Azerbaijan",
    "Bahamas",
    "Bahrain",
    "Bangladesh",
    "Barbados",
    "Belarus",
    "Belgium",
    "Belize",
    "Benin",
    "Bhutan",
    "Bolivia",
    "Bosnia and Herzegovina",
    "Botswana",
    "Brazil",
    "Brunei",
    "Bulgaria",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cambodia",
    "Cameroon",
    "Canada",
    "Central African Republic",
    "Chad",
    "Chile",
    "China",
    "Colombia",
    "Comoros",
    "Democratic Republic of the Congo",
    "Republic of the Congo",
    "Costa Rica",
    "Côte d’Ivoire",
    "Croatia",
    "Cuba",
    "Cyprus",
    "Czech Republic",
    "Denmark",
    "Djibouti",
    "Dominica",
    "Dominican Republic",
    "East Timor",
    "Ecuador",
    "Egypt",
    "El Salvador",
    "Equatorial Guinea",
    "Eritrea",
    "Estonia",
    "Eswatini",
    "Ethiopia",
    "Fiji",
    "Finland",
    "France",
    "Gabon",
    "Gambia",
    "Georgia",
    "Germany",
    "Ghana",
    "Greece",
    "Grenada",
    "Guatemala",
    "Guinea",
    "Guinea-Bissau",
    "Guyana",
    "Haiti",
    "Honduras",
    "Hungary",
    "Iceland",
    "India",
    "Indonesia",
    "Iran",
    "Iraq",
    "Ireland",
    "Israel",
    "Italy",
    "Jamaica",
    "Japan",
    "Jordan",
    "Kazakhstan",
    "Kenya",
    "Kiribati",
    "North Korea",
    "South Korea",
    "Kosovo",
    "Kuwait",
    "Kyrgyzstan",
    "Laos",
    "Latvia",
    "Lebanon",
    "Nicaragua",
    "Niger",
    "Nigeria",
    "North Macedonia",
    "Norway",
    "Oman",
    "Pakistan",
    "Palau",
    "Panama",
    "Papua New Guinea",
    "Paraguay",
    "Peru",
    "Philippines",
    "Poland",
    "Portugal",
    "Qatar",
    "Romania",
    "Russia",
    "Rwanda",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Vincent and the Grenadines",
    "Samoa",
    "San Marino",
    "Sao Tome and Principe",
    "Saudi Arabia",
    "Senegal",
    "Serbia",
    "Seychelles",
    "Sierra Leone",
    "Singapore",
    "Slovakia",
    "Slovenia",
    "Solomon Islands",
    "Somalia",
    "South Africa",
    "Spain",
    "Sri Lanka",
    "Sudan",
    "South Sudan",
    "Suriname",
    "Sweden",
    "Switzerland",
    "Syria",
    "Taiwan",
    "Tajikistan",
    "Tanzania",
    "Thailand",
    "Togo",
    "Tonga",
    "Trinidad and Tobago",
    "Tunisia",
    "Turkey",
    "Turkmenistan",
    "Tuvalu",
    "Uganda",
    "Ukraine",
    "United Arab Emirates",
    "United Kingdom",
    "United States",
    "Uruguay",
    "Uzbekistan",
    "Vanuatu",
    "Vatican City",
    "Venezuela",
    "Vietnam",
    "Yemen",
    "Zambia",
    "Zimbabwe",
]

In [21]:
dict_countries = dict(zip(all_countries, np.arange(1, len(all_countries) + 1)))

## Country mapping

In [22]:
%%time
countries_first_author_2024_du, _ = mapping_countries(
    clean_2024_df_daily_updates.AffiliationFirstAuthor, dict_countries
)

CPU times: user 15min 53s, sys: 7.34 s, total: 16min
Wall time: 16min 2s


## State mapping

In [23]:
all_states = [
    "Alabama",
    "Alaska",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Connecticut",
    "Delaware",
    "Florida",
    "Georgia",
    "Hawaii",
    "Idaho",
    "Illinois",
    "Indiana",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Michigan",
    "Minnesota",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "New York",
    "North Carolina",
    "North Dakota",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Wisconsin",
    "Wyoming",
]

In [24]:
len(all_states)

50

In [25]:
dict_all_states = dict(zip(all_states, np.arange(1, len(all_states) + 1)))

In [26]:
%%time
labels_usa_states_first_author_2024_du, _ = mapping_states(
    clean_2024_df_daily_updates.AffiliationFirstAuthor, dict_all_states
)

CPU times: user 4min 36s, sys: 1.78 s, total: 4min 37s
Wall time: 4min 38s


### Correct USA country with states

In [27]:
countries_first_author_2024_usa_corrected = np.where(
    labels_usa_states_first_author_2024_du != "unknown",
    "United States",
    countries_first_author_2024_du,
)

In [28]:
print(
    "Percentage of US papers before correcting",
    np.sum(countries_first_author_2024_du == "United States")
    / countries_first_author_2024_du.shape[0]
    * 100,
)
print(
    "Percentage of US papers after correcting",
    np.sum(countries_first_author_2024_usa_corrected == "United States")
    / countries_first_author_2024_du.shape[0]
    * 100,
)

Percentage of US papers before correcting 22.45553008707801
Percentage of US papers after correcting 27.090585368574914


# Gender

In this part of the notebook we use the `gender` R package. You can install them in R via comannd `install.packages("gender")`.

## Curate first name lists

### First author

In [10]:
name_first_author = clean_2024_df_daily_updates.NameFirstAuthor

In [12]:
print(
    "Percentge of no names: ",
    np.sum(name_first_author == "") / len(name_first_author) * 100,
)

Percentge of no names:  0.19906999904642061


#### Slicing

In [13]:
%%time
sliced_name_first_author = [
    elem.split()[0] if elem != "" else elem for elem in name_first_author
]

CPU times: user 1.03 s, sys: 17.8 ms, total: 1.05 s
Wall time: 1.05 s


In [14]:
%%time
sliced_name_first_author = [
    elem.split("-")[0] if elem != "" else elem
    for elem in sliced_name_first_author
]

CPU times: user 805 ms, sys: 4.59 ms, total: 809 ms
Wall time: 810 ms


In [15]:
%%time
len_sliced_name_first_author = [len(x) for x in sliced_name_first_author]

CPU times: user 304 ms, sys: 952 µs, total: 305 ms
Wall time: 304 ms


#### Filtering initials

In [22]:
n_initials = len(
    np.array(len_sliced_name_first_author)[
        np.array(len_sliced_name_first_author) == 1
    ]
)
n_total_papers = len(np.array(len_sliced_name_first_author))
print(
    "Percentage of names that are just initials: ",
    n_initials / n_total_papers * 100,
)

Percentage of names that are just initials:  4.865398287019696


In [23]:
print(
    "Percentage of papers with missing names: ",
    np.sum(np.array(sliced_name_first_author) == "")
    / len(sliced_name_first_author)
    * 100,
)

Percentage of papers with missing names:  0.19934480578171387


In [24]:
print(
    "Number of missing names: ",
    len(
        np.array(sliced_name_first_author)[
            np.array(sliced_name_first_author) == ""
        ]
    ),
)

Number of missing names:  7254


In [25]:
%%time
filtered_name_first_author = np.where(
    np.array(len_sliced_name_first_author) == 1, "", sliced_name_first_author
)

CPU times: user 765 ms, sys: 355 µs, total: 766 ms
Wall time: 764 ms


In [26]:
n_initials_and_missing = len(
    np.array(filtered_name_first_author)[
        np.array(filtered_name_first_author) == ""
    ]
)
print(
    "Percentage of initials + missing names: ",
    n_initials_and_missing / n_total_papers * 100,
)

Percentage of initials + missing names:  5.06474309280141


### Last author

In [29]:
name_last_author = clean_2024_df_daily_updates.NameLastAuthor

#### Slicing

In [30]:
%%time
sliced_name_last_author = [
    elem.split()[0] if elem != "" else elem for elem in name_last_author
]

CPU times: user 887 ms, sys: 17.8 ms, total: 905 ms
Wall time: 906 ms


In [31]:
%%time
sliced_name_last_author = [
    elem.split("-")[0] if elem != "" else elem
    for elem in sliced_name_last_author
]

CPU times: user 683 ms, sys: 7.79 ms, total: 691 ms
Wall time: 691 ms


In [32]:
%%time
len_sliced_name_last_author = [len(x) for x in sliced_name_last_author]

CPU times: user 269 ms, sys: 4.79 ms, total: 273 ms
Wall time: 273 ms


#### Filtering initials

In [33]:
n_initials = len(
    np.array(len_sliced_name_last_author)[
        np.array(len_sliced_name_last_author) == 1
    ]
)
n_total_papers = len(np.array(len_sliced_name_last_author))
print(
    "Percentage of names that are just initials: ",
    n_initials / n_total_papers * 100,
)

Percentage of names that are just initials:  5.3387254078887665


In [34]:
print(
    "Number of missing names: ",
    len(
        np.array(sliced_name_last_author)[
            np.array(sliced_name_last_author) == ""
        ]
    ),
)

Number of missing names:  56531


In [35]:
%%time
filtered_name_last_author = np.where(
    np.array(len_sliced_name_last_author) == 1, "", sliced_name_last_author
)

CPU times: user 919 ms, sys: 2.19 ms, total: 921 ms
Wall time: 921 ms


In [36]:
n_initials_and_missing = len(
    np.array(filtered_name_last_author)[
        np.array(filtered_name_last_author) == ""
    ]
)
print(
    "Percentage of initials + missing names: ",
    n_initials_and_missing / n_total_papers * 100,
)

Percentage of initials + missing names:  6.892235363174963


## Gender prediction

In [38]:
%load_ext rpy2.ipython

In [39]:
%R require(gender)
%R require(tibble)

1


In [43]:
date_year = clean_2024_df_daily_updates.Date.str.extract("([12]\d\d\d)").values.astype(int)
date_year = np.array([elem[0] for elem in date_year])

### First author

In [52]:
# I initialize a dataframe where I will store year, name and gender
# This will be the dataframe `year_name_gender_first_author_df'

df_subset = pd.DataFrame(
    {"Years": date_year, "Names": filtered_name_first_author}
)
df_subset

,Years,Names
0,1975,
1,1975,
2,1975,
3,1977,
4,1977,
...,...,...
3638916,2024,Mahtab
3638917,2024,Takeo
3638918,2024,Siobhan
3638919,2024,Fahimeh


In [53]:
%%time

# I predict the gender of those names sorting the df by year.
# The problem is that the returned df does not have the same dimensions as the original, since not all the papers have names.
# Therefore, the next two steps are necessary

gender_prediction_df = pd.DataFrame()
unique_years = np.unique(df_subset.Years)

for year in unique_years:
    print("Year: ", year)

    df_grouped_year = df_subset.groupby("Years").get_group(year)
    df_names = df_grouped_year.Names

    # cases for years outside interval
    if year <= 1930:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 1930, method = "ssa")

    if year >= 2012:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 2012, method = "ssa")

    if (year < 2012) & (year > 1930):
        %R library(gender)
        %R -i df_names -i year -o result result = gender(df_names, years = year, method = "ssa")

    gender_prediction_df = pd.concat(
        [gender_prediction_df, result], ignore_index=True
    )

gender_prediction_df

Year:  1928
Year:  1929
Year:  1930
Year:  1931
Year:  1932
Year:  1933
Year:  1934
Year:  1935
Year:  1936
Year:  1937
Year:  1939
Year:  1941
Year:  1945
Year:  1946
Year:  1948
Year:  1953
Year:  1955
Year:  1956
Year:  1957
Year:  1958
Year:  1959
Year:  1960
Year:  1961
Year:  1962
Year:  1963
Year:  1964
Year:  1965
Year:  1966
Year:  1967
Year:  1968
Year:  1969
Year:  1970
Year:  1971
Year:  1972
Year:  1973
Year:  1974
Year:  1975
Year:  1976
Year:  1977
Year:  1978
Year:  1979
Year:  1980
Year:  1981
Year:  1982
Year:  1983
Year:  1984
Year:  1985
Year:  1986
Year:  1987
Year:  1988
Year:  1989
Year:  1990
Year:  1991
Year:  1992
Year:  1993
Year:  1994
Year:  1995
Year:  1996
Year:  1997
Year:  1998
Year:  1999
Year:  2000
Year:  2001
Year:  2002
Year:  2003
Year:  2004
Year:  2005
Year:  2006
Year:  2007
Year:  2008
Year:  2009
Year:  2010
Year:  2011
Year:  2012
Year:  2013
Year:  2014
Year:  2015
Year:  2016
Year:  2017
Year:  2018
Year:  2019
Year:  2020
Year:  2021
Year

,name,proportion_male,proportion_female,gender,year_min,year_max
0,Carl,0.9948,0.0052,male,1959.0,1959.0
1,Charlotte,0.0026,0.9974,female,1959.0,1959.0
2,Horace,1.0000,0.0000,male,1959.0,1959.0
3,Harriet,0.0000,1.0000,female,1960.0,1960.0
4,Horace,0.9875,0.0125,male,1960.0,1960.0
...,...,...,...,...,...,...
2025803,Yi,1.0000,0.0000,male,2012.0,2012.0
2025804,Yi,1.0000,0.0000,male,2012.0,2012.0
2025805,Yuchen,1.0000,0.0000,male,2012.0,2012.0
2025806,Kristin,0.0000,1.0000,female,2012.0,2012.0


In [55]:
%%time

# In here I am creating a dictionary that has a map name-gender for every year, based on the predictions from the dataframe above (because they are year dependent).
# I do some tricks to create mappings for also years that were not predicted, and years outside the available `ssa' intervals (<1930, >2012)

unique_predicted_years = np.unique(gender_prediction_df["year_min"])
gender_maps_years = {}
for year in unique_years:
    if year >= 2012:
        eff_year = 2012
    if year <= 1930:
        eff_year = 1930
    if (year < 2012) & (year > 1930):
        eff_year = year

    if eff_year not in unique_predicted_years:
        closest_year = unique_predicted_years[
            (unique_predicted_years - eff_year).argmin()
        ]

        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(closest_year)

    else:
        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(eff_year)

    gender_map = dict(
        zip(
            gender_prediction_grouped_year.name,
            gender_prediction_grouped_year.gender,
        )
    )

    gender_maps_years[year] = gender_map

gender_maps_years.keys()

CPU times: user 12.3 s, sys: 19.9 ms, total: 12.4 s
Wall time: 12.4 s


dict_keys([1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1939, 1941, 1945, 1946, 1948, 1953, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026])

In [57]:
%%time
# Now, using the name-gender maps created above, I map the names in the original dataframe (df_subset) to their respective genders, saving them in a new column 'Gender'

# here I add a 'Gender' column to the df_subset
df_subset["Gender"] = ["unknown"] * df_subset.shape[0]

for year in unique_years:
    print(year)

    df_subset_year = (
        df_subset.groupby("Years")
        .get_group(year)
        .Names.apply(lambda x: np.vectorize(gender_maps_years[year].get)(x))
    )
    df_subset_year.rename("Gender", inplace=True)
    df_subset.update(df_subset_year)

df_subset

1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1939
1941
1945
1946
1948
1953
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
CPU times: user 1min 33s, sys: 603 ms, total: 1min 34s
Wall time: 1min 34s


,Years,Names,Gender
0,1975,,unknown
1,1975,,unknown
2,1975,,unknown
3,1977,,unknown
4,1977,,unknown
...,...,...,...
3638916,2024,Mahtab,unknown
3638917,2024,Takeo,male
3638918,2024,Siobhan,female
3638919,2024,Fahimeh,unknown


In [58]:
# check that the number of predicted genders matches the length of the df with the predictions above
len(df_subset[df_subset.Gender != "unknown"])

2025808

In [61]:
#  save
gender_first_author = df_subset.Gender.to_numpy(dtype=str)
np.save(variables_path / "gender_first_author", gender_first_author)

### Last author

In [62]:
df_subset = pd.DataFrame(
    {"Years": date_year, "Names": filtered_name_last_author}
)
df_subset

,Years,Names
0,1975,
1,1975,
2,1975,
3,1977,
4,1977,
...,...,...
3638916,2024,Mohammad
3638917,2024,Atsushi
3638918,2024,Clara
3638919,2024,Mehdi


In [63]:
%%time

gender_prediction_df = pd.DataFrame()
unique_years = np.unique(df_subset.Years)

for year in unique_years:
    print(year)
    df_grouped_year = df_subset.groupby("Years").get_group(year)
    df_names = df_grouped_year.Names

    # cases for years outside interval
    if year <= 1930:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 1930, method = "ssa")

    if year >= 2012:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 2012, method = "ssa")

    if (year < 2012) & (year > 1930):
        %R library(gender)
        %R -i df_names -i year -o result result = gender(df_names, years = year, method = "ssa")

    gender_prediction_df = pd.concat(
        [gender_prediction_df, result], ignore_index=True
    )

gender_prediction_df

1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1939
1941
1945
1946
1948
1953
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
CPU times: user 1min 16s, sys: 787 ms, total: 1min 17s
Wall time: 1min 17s


,name,proportion_male,proportion_female,gender,year_min,year_max
0,Frank,0.9954,0.0046,male,1959.0,1959.0
1,Herbert,0.9966,0.0034,male,1959.0,1959.0
2,Joseph,0.9967,0.0033,male,1959.0,1959.0
3,Alexander,0.9888,0.0112,male,1960.0,1960.0
4,Charles,0.9953,0.0047,male,1960.0,1960.0
...,...,...,...,...,...,...
2040112,Yan,1.0000,0.0000,male,2012.0,2012.0
2040113,Zai,0.5270,0.4730,male,2012.0,2012.0
2040114,Zai,0.5270,0.4730,male,2012.0,2012.0
2040115,Kiran,0.7229,0.2771,male,2012.0,2012.0


In [65]:
%%time

unique_predicted_years = np.unique(gender_prediction_df["year_min"])
gender_maps_years = {}
for year in unique_years:
    if year >= 2012:
        eff_year = 2012
    if year <= 1930:
        eff_year = 1930
    if (year < 2012) & (year > 1930):
        eff_year = year

    if eff_year not in unique_predicted_years:
        closest_year = unique_predicted_years[
            (unique_predicted_years - eff_year).argmin()
        ]

        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(closest_year)

    else:
        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(eff_year)

    gender_map = dict(
        zip(
            gender_prediction_grouped_year.name,
            gender_prediction_grouped_year.gender,
        )
    )

    gender_maps_years[year] = gender_map

gender_maps_years.keys()

CPU times: user 12 s, sys: 55.5 ms, total: 12 s
Wall time: 12.1 s


dict_keys([1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1939, 1941, 1945, 1946, 1948, 1953, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026])

In [68]:
%%time

df_subset["Gender"] = ["unknown"] * df_subset.shape[0]

for year in unique_years:
    print(year)

    df_subset_year = (
        df_subset.groupby("Years")
        .get_group(year)
        .Names.apply(lambda x: np.vectorize(gender_maps_years[year].get)(x))
    )
    df_subset_year.rename("Gender", inplace=True)
    df_subset.update(df_subset_year)

df_subset

1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1939
1941
1945
1946
1948
1953
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
CPU times: user 1min 32s, sys: 333 ms, total: 1min 33s
Wall time: 1min 33s


,Years,Names,Gender
0,1975,,unknown
1,1975,,unknown
2,1975,,unknown
3,1977,,unknown
4,1977,,unknown
...,...,...,...
3638916,2024,Mohammad,male
3638917,2024,Atsushi,unknown
3638918,2024,Clara,female
3638919,2024,Mehdi,male


In [69]:
# check that the number of predicted genders matches the length of the df with the predictions above
len(df_subset[df_subset.Gender != "unknown"])

2040117

In [71]:
#  save
gender_last_author = df_subset.Gender.to_numpy(dtype=str)
np.save(variables_path / "gender_last_author_2024", gender_last_author)

# Save dataframe


- PMID
- abstract
- journal
- publication year (update also preprocess-and-count bc the year is being extracted there)
- label
- affiliation country (from the first affiliation of the first author).
- inferred gender of first and last author

In [4]:
variables_path

NameError: name 'variables_path' is not defined

In [ ]:
np.load(variables)

In [2]:
df = clean_2024_df[
    ["PMID", "Title","AbstractText", "Journal", "Date"]
].copy(deep=True)

NameError: name 'clean_2024_df' is not defined

In [3]:
labels_2024_du

NameError: name 'labels_2024_du' is not defined

In [95]:
df["Year"] = df.Date.str.extract("([12]\d\d\d)").values.astype(int)

In [97]:
df.pop("Date")

1          1975 Nov 01
6             1975 Dec
9             1975 Dec
11         1977 May 11
12            1977 Feb
              ...     
4129316    2024 Jun 30
4129317    2024 Jun 29
4129319       2024 Jul
4129320    2024 Jun 30
4129321    2024 Jun 29
Name: Date, Length: 3638921, dtype: object

In [98]:
df["Labels"] = labels_2024_du

In [99]:
df["Countries"] = countries_first_author_2024_usa_corrected

In [100]:
df["InferredGenderFirstAuthor"] = gender_first_author
df["InferredGenderLastAuthor"] = gender_last_author

In [102]:
len(df)

3638921

In [103]:
df = df.groupby(["PMID"], as_index=False).last()

In [108]:
print(f"There are {len(df)} new papers")

There are 1987842 new papers


In [105]:
df.head()

,PMID,AbstractText,Journal,Year,Labels,Countries,InferredGenderFirstAuthor,InferredGenderLastAuthor
0,10021341,Left-right asymmetry in vertebrates is control...,"Development (Cambridge, England)",1999,unlabeled,Germany,unknown,unknown
1,10021363,"In plants, post-transcriptional gene silencing...",Current biology : CB,1999,unlabeled,France,unknown,unknown
2,10021457,It has been controversial whether high water p...,The Journal of clinical investigation,1999,unlabeled,United States,unknown,unknown
3,10021458,Inherited defects in the degradation of glycos...,The Journal of clinical investigation,1999,unlabeled,United States,unknown,unknown
4,10021462,Peanut allergy is a significant IgE-mediated h...,The Journal of clinical investigation,1999,unlabeled,United States,unknown,unknown


In [106]:
%%time
compression_opts = dict(
    method="zip", archive_name="pubmed_daily_updates_2024_v2.csv"
)

df.to_csv(
    data_path / "pubmed_daily_updates_2024_v2.zip",
    index=False,
    compression=compression_opts,
)

tcmalloc: large alloc 3765600256 bytes == 0x8caf1c000 @ 


CPU times: user 3min 30s, sys: 5.29 s, total: 3min 35s
Wall time: 3min 36s
